In [1]:
import os;os.environ['HIVESERVER2_HOST'] = 'bbx.farlightgames.com'
import sys;sys.path.append('../../')

import pendulum
from tqdm import trange
from jinja2 import Template

from config.HIVESERVER2_token import *
from pyutil.pysql.sql import SQLClient

now = pendulum.date(2024, 1, 30)
client = SQLClient('ethancao')
with open('./sql/dataset.sql', 'r') as f:
    temp = Template(f.read())
    
    pbar = trange(2, 12)
    for delta in pbar:
        ds = now.add(days = - delta)
        pbar.set_description(ds.to_date_string())
        # continue
        [
            client.execute_trino_query(s)
            for s in temp.render(ds = ds).split(';')
        ]

2024-01-19: 100%|██████████| 10/10 [12:46<00:00, 76.67s/it]


In [1]:
import os;os.environ['HIVESERVER2_HOST'] = 'bbx.farlightgames.com'
import sys;sys.path.append('../../')
import pendulum
import pandas as pd
from tqdm import tqdm, trange
from cityhash import CityHash32

from config.HIVESERVER2_token import *
from pyutil.pysql.sql import SQLClient

import tensorflow as tf


import pandas as pd

import sys;sys.path.append('../../')

dense_vocab = {
    'r_hit_number' : [0,37,52,65,79,94,110,131,160,204],
    'r_hited_number' : [0,25,34,41,47,54,62,71,84,108],
    'r_headshot_number' : [0,1,3,4,6,7,9,11,14,19],
    'r_headshoted_number' : [0,1,2,2,3,4,5,7,9],
    'r_damage_amount' : [0,444,619,768,930,1105,1316,1578,1954,2556], 
    'r_damage_taken_amount' : [0,394,497,570,641,697,770,862,984,1169],
    'r_kill_number' : [0,1,2,3,4,5,6,8,11], 
    'r_assist_number' : [0,1,2,3,4],
    'r_dead_number' : [0,1,2,3], 
    'r_death_number' : [0,1,2,3],
    'r_ka_per_3min' : [0.053333065290828305,0.5819396952264781,0.8707765252887384,1.132015890606052,1.3997278853044248,1.677375474831049,1.9912202837942263,2.379640123420753,2.9378460382063785], 
    'r_survival_time' : [0, 392,496,555,618,691,764,852,929,1008],
    'r_hit_rate' : [0.14172694809289044,0.1962671555074827,0.23181147113567696,0.2628059253769059,0.2925804733319646,0.3254649310700026,0.362230869508433,0.4163552085422169,0.5181101700409702],
    'r_headshot_rate' : [3.11899816415794E-5,0.005770877578211221,0.010592320691133663,0.014985647680509492,0.019293363869190706,0.024109606455985037,0.030168272799344437,0.03820006451244755,0.05322657625320898],

    'r_save_number' : [0, 1, 2, 3, 4],
    'r_fade_time' : [0,1,2,4,8,16,24,36,48],
    'r_rank_team' : [-4.,0.,3.,8.,16.,24.,30.,40.],
    'r_safearea_rank' : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],

    'r_elo' : list(range(0, 4200, 300)),
    'r_his_battle_number_3d' : [1,2,4,8,16],
    'r_lifespan' : [0, 3, 7, 14, 30, 60,120, 180],
}

cat_vocab = {
    'r_weapon1' : [11001, 12001, 13001, 14001, 15001, 16001, 17001, 18001, 19001, 20001, 21001, 22001, 23001, 24001, 25001, 26001, 27001, 28001, 29001, 30001, 32001],
    'r_weapon2' : [11001, 12001, 13001, 14001, 15001, 16001, 17001, 18001, 19001, 20001, 21001, 22001, 23001, 24001, 25001, 26001, 27001, 28001, 29001, 30001, 32001],
    'r_character_id' : [100002, 100003, 100004, 100005, 100006, 100007, 100008, 100009, 100010, 100011, 100012, 100013, 100014, 100015, 100016, 100017, 100018, 100019, 100020, 100021, 100022, 100023, 100024, 100025, 100026, 100027],
}


inputs = {
    k : tf.keras.layers.Input((5,), dtype=tf.float32, name=k)
    if k not in ['r_elo', 'r_lifespan', 'r_his_battle_number_3d']
    else tf.keras.layers.Input((1,), dtype=tf.int32, name=k)
    for k, v in dense_vocab.items()
}

inputs.update({
    k : tf.keras.layers.Input((5,), dtype=tf.int32, name=k)
    for k, v in cat_vocab.items()
})

In [3]:
def feature_process(x):
    x = {'{}'.format(kv.split('\x03')[0]) : kv.split('\x03')[1].split('\x04') for kv in x.split('\x02')}
    for k in x.keys():
        if k in ['r_weapon1', 'r_weapon2', 'r_character_id']:
            x[k] = ([int(i) for i in x[k] if i!= 'null'] + [-1] * 5)[0:5]
        elif k in ['r_elo', 'r_his_battle_number_3d', 'r_lifespan']:
            x[k] = ([int(i) for i in x[k] if i!= 'null'] + [-1])[0:1]
        else:
            x[k] = ([float(i) for i in x[k] if i[0].isdigit()] + [-1] * 5)[0:5]
    return x

def to_record(raw : pd.DataFrame, file_name : str):
    groups = raw.groupby('battle_id')
    with tf.io.TFRecordWriter(file_name, options = tf.io.TFRecordOptions(compression_type='GZIP')) as writer:
        for batch in tqdm(groups.groups):
            batch = groups.get_group(batch)
            
            example = tf.train.Example()
            feature = example.features.feature
            for name in ['battle_id', 'team_id', 'app_version', 'battle_fight_num', 'is_battle_number_lift', 'is_damage_amount_lift', 'is_ka_number_lift', 'is_survival_time_lift', 'is_retention']:
                feature[name].CopyFrom(
                    tf.train.Feature(int64_list = tf.train.Int64List(value = batch[name].to_list()))
                )
                
            for name, spec in inputs.items():
                if spec.dtype == tf.float32:
                    feature[name].CopyFrom(
                        tf.train.Feature(float_list = tf.train.FloatList(
                            value = [x for xs in batch.feature for x in xs[name]]    
                        ))
                    )
                else:
                    feature[name].CopyFrom(
                        tf.train.Feature(int64_list = tf.train.Int64List(
                            value = [x for xs in batch.feature for x in xs[name]]    
                        ))
                    )
            writer.write(example.SerializeToString())

query = """
    WITH base AS (
        SELECT *
        FROM hive.ml_tmp_db.solarland_battle_sample_240130
        WHERE dt = '{ds}' 
    ) 

    SELECT
        a.battle_id
        , a.open_id
        , a.team_id
        , a.app_version
        , a.feature
        , a.battle_fight_num
        , a.is_battle_number_lift
        , a.is_retention
        , a.is_damage_amount_lift
        , a.is_ka_number_lift
        , a.is_survival_time_lift
    FROM base AS a
    INNER JOIN (
        SELECT battle_id
        FROM base
        WHERE battle_id is not null
        GROUP BY battle_id
        HAVING 
            COUNT(1) >= 2
            AND COUNT_IF(team_id is null) = 0
            AND MIN(is_retention) = 0 
            AND MAX(is_retention) = 1
    ) AS b
    ON a.battle_id = b.battle_id
    -- ORDER BY battle_id
    -- LIMIT 1000
"""

client = SQLClient('ethancao')
now = pendulum.date(2024, 1, 30)

dss = []

bar = trange(2, 12) 
for delta in bar:
    ds = now.add(days = - delta).to_date_string()
    dss.append(ds)
    bar.set_description(ds)
    
    #### continue if do not need to fetch dataset

    client.reset()
    client.execute_trino_query(query.format(ds = ds))
    raw = client.to_pandas()
    raw = raw.assign(
        battle_id = lambda x: x.battle_id.apply(CityHash32),
        open_id = lambda x: x.open_id.fillna('').apply(CityHash32),
        team_id = lambda x: x.team_id.astype(int),
        feature = lambda x: x.feature.apply(feature_process),
        app_version = lambda x: x.app_version.apply(lambda x: int(x[0])),
        battle_fight_num = lambda x: x.battle_fight_num.fillna(-1).astype(int),
    )
    to_record(raw, file_name = f'../../data/tfrecord/solarland/{ds}_mt.tfrecord')
    del raw
    # break
    # raw.to_parquet(f'../../data/raw/solarland/{ds}.parquet', compression='gzip', index = False)

2024-01-19: 100%|██████████| 10/10 [3:51:24<00:00, 1388.41s/it]


In [ ]:
def feature_process(x):
    x = {'{}'.format(kv.split('\x03')[0]) : kv.split('\x03')[1].split('\x04') for kv in x.split('\x02')}
    for k in x.keys():
        if k in ['r_weapon1', 'r_weapon2', 'r_character_id']:
            x[k] = ([int(i) for i in x[k] if i!= 'null'] + [-1] * 5)[0:5]
        elif k in ['r_elo', 'r_his_battle_number_3d', 'r_lifespan']:
            x[k] = ([int(i) for i in x[k] if i!= 'null'] + [-1])[0:1]
        else:
            x[k] = ([float(i) for i in x[k] if i[0].isdigit()] + [-1] * 5)[0:5]
    return x

def to_record(raw : pd.DataFrame, file_name : str):
    groups = raw.groupby('battle_id')
    with tf.io.TFRecordWriter(file_name, options = tf.io.TFRecordOptions(compression_type='GZIP')) as writer:
        for batch in tqdm(groups.groups):
            batch = groups.get_group(batch)
            
            example = tf.train.Example()
            feature = example.features.feature
            for name in ['battle_id', 'team_id', 'app_version', 'is_battle_number_lift', 'is_damage_amount_lift', 'is_ka_number_lift', 'is_survival_time_lift', 'is_retention']:
                feature[name].CopyFrom(
                    tf.train.Feature(int64_list = tf.train.Int64List(value = batch[name].to_list()))
                )
                
            for name, spec in inputs.items():
                if spec.dtype == tf.float32:
                    feature[name].CopyFrom(
                        tf.train.Feature(float_list = tf.train.FloatList(
                            value = [x for xs in batch.feature for x in xs[name]]    
                        ))
                    )
                else:
                    feature[name].CopyFrom(
                        tf.train.Feature(int64_list = tf.train.Int64List(
                            value = [x for xs in batch.feature for x in xs[name]]    
                        ))
                    )
            writer.write(example.SerializeToString())

query = """
    WITH base AS (
        SELECT *
        FROM hive.ml_tmp_db.solarland_battle_sample_240129
        WHERE dt = '{ds}' 
    ) 

    SELECT
        a.battle_id
        , a.open_id
        , a.team_id
        , a.feature
        , a.is_battle_number_lift
        , a.is_retention
        , a.is_damage_amount_lift
        , a.is_ka_number_lift
        , a.is_survival_time_lift
    FROM base AS a
    INNER JOIN (
        SELECT battle_id
        FROM base
        WHERE battle_id is not null
        GROUP BY battle_id
        HAVING 
            COUNT(1) >= 2
            AND COUNT_IF(team_id is null) = 0
            AND MIN(is_retention) = 0 
            AND MAX(is_retention) = 1
    ) AS b
    ON a.battle_id = b.battle_id
    -- ORDER BY battle_id
    -- LIMIT 1000
"""

client = SQLClient('ethancao')
now = pendulum.date(2024, 1, 30)

dss = []

for delta in trange(2, 12):
    ds = now.add(days = - delta).to_date_string()
    dss.append(ds)
    
    #### continue if do not need to fetch dataset
    continue
    client.reset()
    client.execute_trino_query(query.format(ds = ds))
    raw = client.to_pandas()
    raw = raw.assign(
        battle_id = lambda x: x.battle_id.apply(CityHash32),
        open_id = lambda x: x.open_id.fillna('').apply(CityHash32),
        team_id = lambda x: x.team_id.astype(int),
        feature = lambda x: x.feature.apply(feature_process),
        app_version = lambda x: x.app_version.apply(lambda x: int(x[0]))
    )
    to_record(raw, file_name = f'../../data/tfrecord/solarland/{ds}_multitask.tfrecord')
    del raw
    # break
    # raw.to_parquet(f'../../data/raw/solarland/{ds}.parquet', compression='gzip', index = False)

100%|██████████| 10/10 [00:00<?, ?it/s]


In [3]:
import numpy as np
import tensorflow as tf
x = np.random.random(())
y = np.random.randint(0, 3, size=(3, ))

array([[0, 1, 0],
       [0, 0, 1]])

In [4]:
# from cityhash import CityHash32

# def feature_process(x):
#     x = {'{}'.format(kv.split('\x03')[0]) : kv.split('\x03')[1].split('\x04') for kv in x.split('\x02')}
#     for k in x.keys():
#         if k in ['r_weapon1', 'r_weapon2', 'r_character_id']:
#             x[k] = ([int(i) for i in x[k] if i!= 'null'] + [-1] * 5)[0:5]
#         else:
#             x[k] = ([float(i) for i in x[k] if i[0].isdigit()] + [-1] * 5)[0:5]
#     return x

# ds = raw.assign(
#     battle_id = lambda x: x.battle_id.apply(CityHash32),
#     ticket_id = lambda x: x.ticket_id.apply(CityHash32),
#     open_id = lambda x: x.open_id.apply(CityHash32),
#     team_id = lambda x: x.team_id.astype(int),
#     feature = lambda x: x.feature.apply(feature_process),
# )

dense_vocab = {
    'r_hit_number' : [0,37,52,65,79,94,110,131,160,204],
    'r_hited_number' : [0,25,34,41,47,54,62,71,84,108],
    'r_headshot_number' : [0,1,3,4,6,7,9,11,14,19],
    'r_headshoted_number' : [0,1,2,2,3,4,5,7,9],
    'r_damage_amount' : [0,444,619,768,930,1105,1316,1578,1954,2556], 
    'r_damage_taken_amount' : [0,394,497,570,641,697,770,862,984,1169],
    'r_kill_number' : [0,1,2,3,4,5,6,8,11], 
    'r_assist_number' : [0,1,2,3,4],
    'r_dead_number' : [0,1,2,3], 
    'r_death_number' : [0,1,2,3],
    'r_ka_per_3min' : [0.053333065290828305,0.5819396952264781,0.8707765252887384,1.132015890606052,1.3997278853044248,1.677375474831049,1.9912202837942263,2.379640123420753,2.9378460382063785], 
    'r_survival_time' : [0, 392,496,555,618,691,764,852,929,1008],
    'r_hit_rate' : [0.14172694809289044,0.1962671555074827,0.23181147113567696,0.2628059253769059,0.2925804733319646,0.3254649310700026,0.362230869508433,0.4163552085422169,0.5181101700409702],
    'r_headshot_rate' : [3.11899816415794E-5,0.005770877578211221,0.010592320691133663,0.014985647680509492,0.019293363869190706,0.024109606455985037,0.030168272799344437,0.03820006451244755,0.05322657625320898],

    'r_elo' : list(range(0, 4200, 300)),
    'r_his_battle_number_3d' : [1,2,4,8,16],
    'r_lifespan' : [0, 3, 7, 14, 30, 60,120, 180],
}

cat_vocab = {
    'r_weapon1' : [11001, 12001, 13001, 14001, 15001, 16001, 17001, 18001, 19001, 20001, 21001, 22001, 23001, 24001, 25001, 26001, 27001, 28001, 29001, 30001, 32001],
    'r_weapon2' : [11001, 12001, 13001, 14001, 15001, 16001, 17001, 18001, 19001, 20001, 21001, 22001, 23001, 24001, 25001, 26001, 27001, 28001, 29001, 30001, 32001],
    'r_character_id' : [100002, 100003, 100004, 100005, 100006, 100007, 100008, 100009, 100010, 100011, 100012, 100013, 100014, 100015, 100016, 100017, 100018, 100019, 100020, 100021, 100022, 100023, 100024, 100025, 100026, 100027],
}

vocab = dict(
    dense_vocab = dense_vocab,
    cat_vocab = cat_vocab
)

In [3]:
## tensorflow 2.11

# import tensorflow as tf
# from tfutils.layers.vsn import FieldVariableSelectionWithContext
# from tfutils.layers.attn import BatchNormTransformerEncoder

# def get_input(inputs, embeddings, key : str, hidden_size : int = 8):
#     dtype = (
#         tf.int32 
#         if key in ['r_elo', 'r_his_battle_number_3d', 'r_lifespan', 'r_weapon1', 'r_weapon2', 'r_character_id'] 
#         else tf.float32
#     )
#     shape = (
#         (1,)
#         if key in ['r_elo', 'r_his_battle_number_3d', 'r_lifespan'] 
#         else (5,)
#     )
    
#     x = inputs[key] = tf.keras.layers.Input(shape=shape, name=key, dtype=dtype)
#     if key in dense_vocab:
#         v = dense_vocab[key]
#         x = tf.clip_by_value(x, min(v) - 1, max(v) + 1)
#         x = tf.keras.layers.Discretization(v)(x)
#         x = tf.keras.layers.Embedding(len(v) + 1, hidden_size)(x)
#     else:
#         v = cat_vocab[key]
#         x = tf.keras.layers.IntegerLookup(vocabulary=v, oov_token=0)(x)
#         x = tf.keras.layers.Embedding(len(v) + 1, hidden_size)(x)
#     embeddings[key] = x
    

# inputs, embeddings = dict(), dict()
# HIDDEN_SIZE = 8    

# for key in dense_vocab.keys():
#     get_input(inputs, embeddings, key, hidden_size=HIDDEN_SIZE)
# for key in cat_vocab.keys():
#     get_input(inputs, embeddings, key, hidden_size=HIDDEN_SIZE)
    

# embeddings = tf.concat(embeddings.values(), axis = -2)
# context = tf.math.reduce_mean(embeddings, axis = 0, keepdims=True)
# embeddings = BatchNormTransformerEncoder(8, 4, 8, 0.1)(embeddings)
# embeddings = FieldVariableSelectionWithContext(embeddings.shape[1], 32, 0.1)(embeddings, context)


# team_id = tf.keras.layers.Input((1,), dtype=tf.int32, name='team_id')
# ticket_id = tf.keras.layers.Input((1,), dtype=tf.int32, name='ticket_id')
# teammate_matrix = tf.cast(team_id == tf.transpose(team_id), tf.float32)


In [31]:
from itertools import chain
from typing import Dict, List, Tuple, Union

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchmetrics
import lightning as L

from cityhash import CityHash32
from torchnlp.encoders import LabelEncoder

__FEATURE_KEY_GENERATOR = list(chain(vocab['dense_vocab'].keys().__iter__(), vocab['cat_vocab'].keys().__iter__()))
def process_feature(feature : dict):
    return {
        key : torch.Tensor([x[key] for x in feature]).to('cuda')
        for key in __FEATURE_KEY_GENERATOR
    }
        

class MatchmakingInputLayer(nn.Module):
    def __init__(self, hidden_size : int = 8, vocab : Dict = None):
        super().__init__()
        self.vocab = {
            'dense_vocab' : {
                k : torch.Tensor(v).to('cuda')
                for k, v in vocab['dense_vocab'].items()
            },
            'cat_vocab' : {
                k : torch.Tensor(v).to('cuda')
                for k, v in vocab['cat_vocab'].items()
            } 
        }
        self.embeddings = {
            key : nn.Embedding(len(value) + 1, hidden_size)
            for key, value in chain(vocab['dense_vocab'].items(), vocab['cat_vocab'].items())
        }
        
        self.token_encoder = {
            key : LabelEncoder(sample = value, unknown_index = 0)
            for key, value in vocab['cat_vocab'].items()
        }
        
        slot, self.slot = 0, dict()
        for key, value in chain(vocab['dense_vocab'].items(), vocab['cat_vocab'].items()):
            self.slot[key], slot = slot, slot + len(value) 
        self.max_slot = slot
        
        self.embeddings = nn.Embedding(self.max_slot, hidden_size)
    
    
    def forward(self, feature):
        x = [
            torch.bucketize(feature[key], self.vocab['dense_vocab'][key])  + self.slot[key]
            for key in vocab['dense_vocab'].keys()
        ] + [
            self.token_encoder[key].batch_encode(feature[key]).view(-1, 1).to('cuda') + self.slot[key]
            for key in vocab['cat_vocab'].keys()
        ]
        x = torch.concat(x, axis = -1)
        x = self.embeddings(x)
        return x

In [6]:
class TimeDistributed(nn.Module):
    def __init__(self, module: nn.Module, batch_first: bool = False):
        super().__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)

        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size)

        y = self.module(x_reshape)

        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)
        return y


class TimeDistributedInterpolation(nn.Module):
    def __init__(self, output_size: int, batch_first: bool = False, trainable: bool = False):
        super().__init__()
        self.output_size = output_size
        self.batch_first = batch_first
        self.trainable = trainable
        if self.trainable:
            self.mask = nn.Parameter(torch.zeros(self.output_size, dtype=torch.float32))
            self.gate = nn.Sigmoid()

    def interpolate(self, x):
        upsampled = F.interpolate(x.unsqueeze(1), self.output_size, mode="linear", align_corners=True).squeeze(1)
        if self.trainable:
            upsampled = upsampled * self.gate(self.mask.unsqueeze(0)) * 2.0
        return upsampled

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.interpolate(x)

        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size)

        y = self.interpolate(x_reshape)

        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)

        return y


class GatedLinearUnit(nn.Module):
    """Gated Linear Unit"""

    def __init__(self, input_size: int, hidden_size: int = None, dropout: float = None):
        super().__init__()

        if dropout is not None:
            self.dropout = nn.Dropout(dropout)
        else:
            self.dropout = dropout
        self.hidden_size = hidden_size or input_size
        self.fc = nn.Linear(input_size, self.hidden_size * 2)

        self.init_weights()

    def init_weights(self):
        for n, p in self.named_parameters():
            if "bias" in n:
                torch.nn.init.zeros_(p)
            elif "fc" in n:
                torch.nn.init.xavier_uniform_(p)

    def forward(self, x):
        if self.dropout is not None:
            x = self.dropout(x)
        x = self.fc(x)
        x = F.glu(x, dim=-1)
        return x


class ResampleNorm(nn.Module):
    def __init__(self, input_size: int, output_size: int = None, trainable_add: bool = True):
        super().__init__()

        self.input_size = input_size
        self.trainable_add = trainable_add
        self.output_size = output_size or input_size

        if self.input_size != self.output_size:
            self.resample = TimeDistributedInterpolation(self.output_size, batch_first=True, trainable=False)

        if self.trainable_add:
            self.mask = nn.Parameter(torch.zeros(self.output_size, dtype=torch.float))
            self.gate = nn.Sigmoid()
        self.norm = nn.LayerNorm(self.output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.input_size != self.output_size:
            x = self.resample(x)

        if self.trainable_add:
            x = x * self.gate(self.mask) * 2.0

        output = self.norm(x)
        return output


class AddNorm(nn.Module):
    def __init__(self, input_size: int, skip_size: int = None, trainable_add: bool = True):
        super().__init__()

        self.input_size = input_size
        self.trainable_add = trainable_add
        self.skip_size = skip_size or input_size

        if self.input_size != self.skip_size:
            self.resample = TimeDistributedInterpolation(self.input_size, batch_first=True, trainable=False)

        if self.trainable_add:
            self.mask = nn.Parameter(torch.zeros(self.input_size, dtype=torch.float))
            self.gate = nn.Sigmoid()
        self.norm = nn.LayerNorm(self.input_size)

    def forward(self, x: torch.Tensor, skip: torch.Tensor):
        if self.input_size != self.skip_size:
            skip = self.resample(skip)

        if self.trainable_add:
            skip = skip * self.gate(self.mask) * 2.0

        output = self.norm(x + skip)
        return output


class GateAddNorm(nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_size: int = None,
        skip_size: int = None,
        trainable_add: bool = False,
        dropout: float = None,
    ):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size or input_size
        self.skip_size = skip_size or self.hidden_size
        self.dropout = dropout

        self.glu = GatedLinearUnit(self.input_size, hidden_size=self.hidden_size, dropout=self.dropout)
        self.add_norm = AddNorm(self.hidden_size, skip_size=self.skip_size, trainable_add=trainable_add)

    def forward(self, x, skip):
        output = self.glu(x)
        output = self.add_norm(output, skip)
        return output


class GatedResidualNetwork(nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_size: int,
        output_size: int,
        dropout: float = 0.1,
        context_size: int = None,
        residual: bool = False,
    ):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.context_size = context_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.residual = residual

        if self.input_size != self.output_size and not self.residual:
            residual_size = self.input_size
        else:
            residual_size = self.output_size

        if self.output_size != residual_size:
            self.resample_norm = ResampleNorm(residual_size, self.output_size)

        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.elu = nn.ELU()

        if self.context_size is not None:
            self.context = nn.Linear(self.context_size, self.hidden_size, bias=False)

        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.init_weights()

        self.gate_norm = GateAddNorm(
            input_size=self.hidden_size,
            skip_size=self.output_size,
            hidden_size=self.output_size,
            dropout=self.dropout,
            trainable_add=False,
        )
    def init_weights(self):
        for name, p in self.named_parameters():
            if "bias" in name:
                torch.nn.init.zeros_(p)
            elif "fc1" in name or "fc2" in name:
                torch.nn.init.kaiming_normal_(p, a=0, mode="fan_in", nonlinearity="leaky_relu")
            elif "context" in name:
                torch.nn.init.xavier_uniform_(p)

    def forward(self, x, context=None, residual=None):
        if residual is None:
            residual = x

        if self.input_size != self.output_size and not self.residual:
            residual = self.resample_norm(residual)

        x = self.fc1(x)
        if context is not None:
            context = self.context(context)
            x = x + context
        x = self.elu(x)
        x = self.fc2(x)
        x = self.gate_norm(x, residual)
        return x



class FactorizationMachine(nn.Module):
    def __init__(
        self,
    ) -> None:
        super().__init__()

    def forward(
        self,
        fm_input: List[torch.Tensor],
    ) -> torch.Tensor:
        sum_of_input = torch.sum(fm_input, dim=1, keepdim=True)
        sum_of_square = torch.sum(fm_input * fm_input, dim=1, keepdim=True)
        square_of_sum = sum_of_input * sum_of_input
        cross_term = square_of_sum - sum_of_square
        cross_term = torch.sum(cross_term, dim=1, keepdim=True) * 0.5  # [B, 1]
        return cross_term
    


In [35]:
class MatchMakingDataLoader(L.LightningDataModule):
    __FEATURE_KEY_GENERATOR = ['r_hit_number','r_hited_number','r_headshot_number','r_headshoted_number','r_damage_amount','r_damage_taken_amount','r_kill_number','r_assist_number','r_dead_number','r_death_number','r_ka_per_3min','r_survival_time','r_hit_rate','r_headshot_rate','r_elo','r_his_battle_number_3d','r_lifespan','r_weapon1','r_weapon2','r_character_id',]
    
    def __init__(self, parquet_file_path : str):
        super().__init__()
        self.parquet_file_path = parquet_file_path
        self.data = None
        self.index = {}
        self.setup()
    
    def list2array(self, xs):
        return torch.Tensor([x for x in xs])
    
    @staticmethod    
    def feature_process(x):
        x = {'{}'.format(kv.split('\x03')[0]) : kv.split('\x03')[1].split('\x04') for kv in x.split('\x02')}
        for k in x.keys():
            if k in ['r_weapon1', 'r_weapon2', 'r_character_id']:
                x[k] = ([int(i) for i in x[k] if i!= 'null'] + [-1] * 5)[0:5]
            else:
                x[k] = ([float(i) for i in x[k] if i[0].isdigit()] + [-1] * 5)[0:5]
        return x
    
    def setup(self, **kwargs):
        
        raw = pd.read_parquet(self.parquet_file_path)
        raw = raw.assign(
            battle_id = lambda x: x.battle_id.apply(CityHash32),
            ticket_id = lambda x: x.ticket_id.apply(CityHash32),
            open_id = lambda x: x.open_id.apply(CityHash32),
            team_id = lambda x: x.team_id.astype(int),
            feature = lambda x: x.feature.apply(self.feature_process),
        )
        self.data = [
            {
                key : np.array(x[key]) if key not in self.__FEATURE_KEY_GENERATOR else np.array(x['feature'][key])
                for key in ['ticket_id', 'team_id', 'is_churn'] + self.__FEATURE_KEY_GENERATOR
            }      
            for x in raw.to_dict('records')
        ] 
        self.index['fit'] = [idx.to_list() for idx in raw.query('battle_id % 10 < 7').groupby('battle_id').groups.values()]     
        self.index['test'] = [idx.to_list() for idx in raw.query('battle_id % 10 < 7').groupby('battle_id').groups.values()]     
        
            
        
    def train_dataloader(self):
        return DataLoader(
            dataset = self.data,
            batch_sampler=self.index['fit']
        )
    
    def val_dataloader(self):
        return DataLoader(
            dataset = self.data,
            batch_sampler=self.index['test']
        )
    
    def test_dataloader(self):
        pass
    

class MatchMakingModel(L.LightningModule):
    def __init__(self, hidden_size : int = 8):
        super().__init__()  
        self.encoder = nn.Sequential(MatchmakingInputLayer(hidden_size=hidden_size, vocab = vocab), nn.TransformerEncoderLayer(hidden_size, 4, hidden_size, 0.1))
        self.context_gate_after_encoder = GatedResidualNetwork(88 * hidden_size, hidden_size, 88, dropout=0.1, context_size=hidden_size)
        self.output_fm = FactorizationMachine()
        self.output_linear = nn.Linear(2 * hidden_size, 1)
        
        
        self.acc = torchmetrics.classification.BinaryAccuracy()
        self.auc = torchmetrics.classification.BinaryAUROC()
    
    def forward(self, batch):
        x = self.encoder(batch)
        context = nn.Flatten()(x).mean(axis = 0, keepdim = True)
        
        context = self.context_gate_after_encoder(context)
        context = F.softmax(context, dim = -1)
        x = (context.unsqueeze(-1) * x).sum(axis = 1)
        
        size = batch['team_id'].shape[0]
        m00 = torch.eye(size, device = 'cuda')
        m0 = m00.repeat([1, size]).view(-1, size)
        m1 = m00.repeat([size, 1]).view(-1, size)

        y = torch.concat([m0 @ x, m1 @ x], axis = -1)
        y = self.output_fm(y) + self.output_linear(y)
        y = F.sigmoid(y).view((size, size))
        return y
        
    def training_step(self, batch, batch_idx):
        team_id, true = batch['team_id'], batch['is_churn']
        y = self.forward(batch)
        
        masking = team_id.view(-1,1)
        masking = (team_id == team_id.T).float()
        
        size = team_id.shape[0]
        true = true.view(-1,1).repeat([1, size]).float()
        loss = F.binary_cross_entropy(y, true, weight=masking)
        
        self.acc(y, true)
        self.log('train_acc', self.acc, on_step=True, on_epoch=True)
        
        self.auc(y, true)
        self.log('train_auc', self.auc, on_step=True, on_epoch=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        team_id, true = batch['team_id'], batch['is_churn']
        y = self.forward(batch)
        
        masking = team_id.view(-1,1)
        masking = (team_id == team_id.T).float()
        
        size = team_id.shape[0]
        true = true.view(-1,1).repeat([1, size]).float()
        loss = F.binary_cross_entropy(y, true, weight=masking)
        
        self.acc(y, true)
        self.log('valid_acc', self.acc, on_step=True, on_epoch=True)
        
        self.auc(y, true)
        self.log('valid_auc', self.auc, on_step=True, on_epoch=True)
        
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
        

In [28]:
dataset = MatchMakingDataLoader('../../data/raw/farlight84-train-set-240115.pqt')

In [37]:
torch.set_float32_matmul_precision('medium')
trainer = L.Trainer(
    accelerator='gpu',
    max_epochs=3,
)
model = MatchMakingModel(hidden_size=8)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
model.train()
trainer.fit(model, datamodule=dataset)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                       | Type                 | Params
--------------------------------------------------------------------
0 | encoder                    | Sequential           | 2.2 K 
1 | context_gate_after_encoder | GatedResidualNetwork | 7.8 K 
2 | output_fm                  | FactorizationMachine | 0     
3 | output_linear              | Linear               | 17    
4 | acc                        | BinaryAccuracy       | 0     
5 | auc                        | BinaryAUROC          | 0     
--------------------------------------------------------------------
10.0 K    Trainable params
0         Non-trainable params
10.0 K    Total params
0.040     Total estimated model params size (MB)


d:\Anaconda\envs\torch\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
d:\Anaconda\envs\torch\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
d:\Anaconda\envs\torch\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0:   2%|▏         | 102/5109 [00:01<01:21, 61.48it/s, v_num=16]

d:\Anaconda\envs\torch\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 0: 100%|██████████| 5109/5109 [02:34<00:00, 33.15it/s, v_num=16]

d:\Anaconda\envs\torch\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
d:\Anaconda\envs\torch\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAUROC was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028


ValueError: No samples to concatenate

In [30]:
trainer.callback_metrics

{}